In [ ]:
@file:DependsOn("/data/tools/actin-personalization/actin-personalization.jar")

import com.hartwig.actin.personalization.datamodel.ReferencePatient
import com.hartwig.actin.personalization.ncr.datamodel.NcrRecord
import com.hartwig.actin.personalization.ncr.interpretation.ReferencePatientFactory
import com.hartwig.actin.personalization.ncr.serialization.NcrDataReader

val records = NcrDataReader.read("/data/patient_like_me/ncr/latest/K2400223.csv")
val patients = ReferencePatientFactory.default().create(records)

In [ ]:
val tumors = patients.flatMap { it.tumorEntries }

In [ ]:
import com.hartwig.actin.personalization.datamodel.NumberOfCciCategories
import com.hartwig.actin.personalization.datamodel.AnorectalVergeDistanceCategory

data class TumorSummary(
    val cci: Map<Int?, Int> = emptyMap(),
    val cciNumberOfCategories: Map<NumberOfCciCategories?, Int> = emptyMap(),
    val cciHasAids: Map<Boolean?, Int> = emptyMap(),
    val cciHasCongestiveHeartFailure: Map<Boolean?, Int> = emptyMap(),
    val cciHasCollagenosis: Map<Boolean?, Int> = emptyMap(),
    val cciHasCopd: Map<Boolean?, Int> = emptyMap(),
    val cciHasCerebrovascularDisease: Map<Boolean?, Int> = emptyMap(),
    val cciHasDementia: Map<Boolean?, Int> = emptyMap(),
    val cciHasDiabetesMellitus: Map<Boolean?, Int> = emptyMap(),
    val cciHasDiabetesMellitusWithEndOrganDamage: Map<Boolean?, Int> = emptyMap(),
    val cciHasOtherMalignancy: Map<Boolean?, Int> = emptyMap(),
    val cciHasOtherMetastaticSolidTumor: Map<Boolean?, Int> = emptyMap(),
    val cciHasMyocardialInfarct: Map<Boolean?, Int> = emptyMap(),
    val cciHasMildLiverDisease: Map<Boolean?, Int> = emptyMap(),
    val cciHasHemiplegiaOrParaplegia: Map<Boolean?, Int> = emptyMap(),
    val cciHasPeripheralVascularDisease: Map<Boolean?, Int> = emptyMap(),
    val cciHasRenalDisease: Map<Boolean?, Int> = emptyMap(),
    val cciHasLiverDisease: Map<Boolean?, Int> = emptyMap(),
    val cciHasUlcerDisease: Map<Boolean?, Int> = emptyMap(),

    val presentedWithIleus: Map<Boolean?, Int> = emptyMap(),
    val presentedWithPerforation: Map<Boolean?, Int> = emptyMap(),
    val anorectalVergeDistanceCategory: Map<AnorectalVergeDistanceCategory?, Int> = emptyMap(),

    val hasMsi: Map<Boolean?, Int> = emptyMap(),
    val hasBrafMutation: Map<Boolean?, Int> = emptyMap(),
    val hasBrafV600EMutation: Map<Boolean?, Int> = emptyMap(),
    val hasRasMutation: Map<Boolean?, Int> = emptyMap(),
    val hasKrasG12CMutation: Map<Boolean?, Int> = emptyMap(),
)

fun <T> updatedMap(oldMap: Map<T, Int>, instance: T) = 
    oldMap + mapOf(instance to oldMap.getOrDefault(instance, 0) + 1)
    
val summary = tumors.fold(TumorSummary()) { acc, tumor ->
    val x = tumor.diagnosis
    TumorSummary(
        updatedMap(acc.cci, x.cci),
        updatedMap(acc.cciNumberOfCategories, x.cciNumberOfCategories),
        updatedMap(acc.cciHasAids, x.cciHasAids),
        updatedMap(acc.cciHasCongestiveHeartFailure, x.cciHasCongestiveHeartFailure),
        updatedMap(acc.cciHasCollagenosis, x.cciHasCollagenosis),
        updatedMap(acc.cciHasCopd, x.cciHasCopd),
        updatedMap(acc.cciHasCerebrovascularDisease, x.cciHasCerebrovascularDisease),
        updatedMap(acc.cciHasDementia, x.cciHasDementia),
        updatedMap(acc.cciHasDiabetesMellitus, x.cciHasDiabetesMellitus),
        updatedMap(acc.cciHasDiabetesMellitusWithEndOrganDamage, x.cciHasDiabetesMellitusWithEndOrganDamage),
        updatedMap(acc.cciHasOtherMalignancy, x.cciHasOtherMalignancy),
        updatedMap(acc.cciHasOtherMetastaticSolidTumor, x.cciHasOtherMetastaticSolidTumor),
        updatedMap(acc.cciHasMyocardialInfarct, x.cciHasMyocardialInfarct),
        updatedMap(acc.cciHasMildLiverDisease, x.cciHasMildLiverDisease),
        updatedMap(acc.cciHasHemiplegiaOrParaplegia, x.cciHasHemiplegiaOrParaplegia),
        updatedMap(acc.cciHasPeripheralVascularDisease, x.cciHasPeripheralVascularDisease),
        updatedMap(acc.cciHasRenalDisease, x.cciHasRenalDisease),
        updatedMap(acc.cciHasLiverDisease, x.cciHasLiverDisease),
        updatedMap(acc.cciHasUlcerDisease, x.cciHasUlcerDisease),
        updatedMap(acc.presentedWithIleus, x.presentedWithIleus),
        updatedMap(acc.presentedWithPerforation, x.presentedWithPerforation),
        updatedMap(acc.anorectalVergeDistanceCategory, x.anorectalVergeDistanceCategory),
        updatedMap(acc.hasMsi, x.hasMsi),
        updatedMap(acc.hasBrafMutation, x.hasBrafMutation),
        updatedMap(acc.hasBrafV600EMutation, x.hasBrafV600EMutation),
        updatedMap(acc.hasRasMutation, x.hasRasMutation),
        updatedMap(acc.hasKrasG12CMutation, x.hasKrasG12CMutation),
    )
}
summary.toString().replace("}, ", "},\n")

In [ ]:
import com.hartwig.actin.personalization.datamodel.MetastasesDetectionStatus
import com.hartwig.actin.personalization.datamodel.Episode
import com.hartwig.actin.personalization.datamodel.Treatment

fun Episode.doesNotIncludeAdjuvantOrNeoadjuvantTreatment(): Boolean {
    return !hasHadPreSurgerySystemicChemotherapy &&
        !hasHadPostSurgerySystemicChemotherapy &&
        !hasHadPreSurgerySystemicTargetedTherapy &&
        !hasHadPostSurgerySystemicTargetedTherapy
}

val referencePop = tumors.map { (diagnosis, episodes) -> diagnosis to episodes.single { it.order == 1 } }
    .filter { (_, episode) ->
        episode.distantMetastasesDetectionStatus == MetastasesDetectionStatus.AT_START &&
        episode.systemicTreatmentPlan != null &&
        episode.systemicTreatmentPlan!!.treatment != Treatment.OTHER &&
        episode.doesNotIncludeAdjuvantOrNeoadjuvantTreatment()
    }

In [ ]:
%use dataframe(0.12.1)

import com.hartwig.actin.personalization.datamodel.LocationGroup
import com.hartwig.actin.personalization.datamodel.Diagnosis

typealias DiagnosisAndEpisode = Pair<Diagnosis, Episode>

fun pfsForPopulation(population: List<DiagnosisAndEpisode>): String {
    val medianPfs = median(population.mapNotNull { (_, episode) -> episode.systemicTreatmentPlan?.observedPfsDays })
    return "$medianPfs (n=${population.size})"
}

fun pfsTable(patientsByTreatment: Map<Treatment?, List<DiagnosisAndEpisode>>, columnDefinitions: List<Pair<String, (DiagnosisAndEpisode) -> Boolean>>): DataFrame<*> {
    val sortedPatients = patientsByTreatment.entries.map { (treatment, patients) ->
        treatment to patients.filter { (_, episode) -> episode.systemicTreatmentPlan?.observedPfsDays != null }
    }
        .sortedByDescending { it.second.size }
                             
    val entries = columnDefinitions.map { (title, criteria) ->
        val populationSize: Int = sortedPatients.flatMap { it.second }.count(criteria)
        val annotatedTitle = "$title (n=$populationSize)"
        annotatedTitle to sortedPatients.map { pfsForPopulation(it.second.filter(criteria)) }
    }
    
    return dataFrame(sortedPatients.map { it.first }, "Treatment", entries)
}

fun treatmentDecisionForSubPopulation(subPopulationSize: Int, populationSize: Int): String {
    return String.format("%.1f%%", 100.0 * subPopulationSize / populationSize)
}

fun treatmentDecisionTable(
    patientsByTreatment: Map<Treatment?, List<DiagnosisAndEpisode>>,
    columnDefinitions: List<Pair<String, (DiagnosisAndEpisode) -> Boolean>>
): DataFrame<*> {
    val allPatients = patientsByTreatment.flatMap { it.value }
    val sortedPatients = patientsByTreatment.entries.sortedByDescending { it.value.size }
                             
    val entries = columnDefinitions.map { (title, criteria) ->
        val populationSize: Int = allPatients.count(criteria)
        val annotatedTitle = "$title (n=$populationSize)"
        val entries = sortedPatients.map { (_, patients) ->
            treatmentDecisionForSubPopulation(patients.count(criteria), populationSize) }
        annotatedTitle to entries
    }
    
    return dataFrame(sortedPatients.map { it.key }, "Treatment", entries)
}

fun dataFrame(rowLabels: List<Any?>, firstColumnName: String, namedColumns: List<Pair<String, List<String>>>): DataFrame<*> {
    val labelStrings = rowLabels.map { it?.let { it.toString() } ?: "None" }
    return (listOf(firstColumnName to labelStrings) + namedColumns).toMap().toDataFrame()
}

fun median(list: List<Int>): Double {
    return when(list.size) {
        0 -> Double.NaN
        1 -> list.first().toDouble()
        else -> {
            val midPoint = list.size / 2
            list.sorted().let {
                if (it.size % 2 == 0)
                    (it[midPoint] + it[midPoint - 1]) / 2.0
                else
                    it[midPoint].toDouble()
            }
        }
    }
}

val patientsByTreatment = referencePop
    // .filter { it.first.hasMsi == true }
    .groupBy { (_, episode) -> episode.systemicTreatmentPlan?.treatment }

fun columnDefinitionsForCriteria(
    whoStatus: Int, age: Int, hasRasMutation: Boolean, metastasisLocationGroups: Set<LocationGroup>
): List<Pair<String, (DiagnosisAndEpisode) -> Boolean>> {
    val minAge = age - 5
    val maxAge = age + 5
    return listOf(
        "All" to { true },
        "WHO=$whoStatus" to { it.second.whoStatusPreTreatmentStart == whoStatus },
        "Age $minAge-${maxAge}y" to { it.first.ageAtDiagnosis in minAge..maxAge },
        "KRAS mut" to { it.first.hasRasMutation == hasRasMutation },
        "${metastasisLocationGroups.joinToString("/")} metastases" to
            { it.second.metastases.map { meta -> meta.location.locationGroup }.toSet() == metastasisLocationGroups }
    )
}

val columnDefinitions = columnDefinitionsForCriteria(1, 50, true, setOf(LocationGroup.PERITONEUM))

pfsTable(patientsByTreatment, columnDefinitions)

In [ ]:
fun isYear(y: Int): (DiagnosisAndEpisode) -> Boolean = { it.second.tumorIncidenceYear == y }

val columnDefinitions = (2015..2022).toList().map { y ->
    y.toString() to isYear(y)
}
treatmentDecisionTable(patientsByTreatment, columnDefinitions)

In [ ]:
patients.flatMap { it.tumorEntries }.groupingBy { it.diagnosis.hasHadTumorDirectedSystemicTherapy }.eachCount()